In [5]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.3 MB/s eta 0:00:00


In [ ]:
import openai
import json

from dotenv import load_dotenv
load_dotenv()
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

def summarize_text(prompt, transcript, captions):
    prompt += "\n\nTranscript - " + transcript + "\n\nCaptions - " + captions

    response = openai.Completion.create(
        engine="davinci-002",
        prompt=prompt,
        max_tokens=20,
        temperature=0.5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].text.strip()

def summarize_json_file(input_file):
    with open(input_file, 'r+') as f:
        data = json.load(f)

        for item in data:
            prompt ='''You will be given image captions from previous frames of a clip.
            You will also be given the transcript of the current clip, a list of identified actions that may be being performed in that clip, and the identified person in the current frame.
            The generated image caption and identified action may be partially or fully incorrect.
            If you feel the data is incorrect, do not include it in the output.
            If data is missing, it will say 'None'.
            Your job is to create a great caption or explanation for the current frame while keeping in mind the context provided by the previous captions, transcript, and identified person.'''
            transcript = item.get('text', 'None')
            captions = item.get('merged_caption', 'None')
            summary = summarize_text(prompt, transcript, captions)
            item['summary'] = summary
            del item['text']
            del item['merged_caption']

        f.seek(0)
        json.dump(data, f, indent=4)
        f.truncate()

# Replace 'input.json' with your input file path
summarize_json_file('/content/transcripts.json')


In [1]:
import json
import math




def format_seconds(seconds):
    return f"{math.floor(seconds)}"


def read_json_file(filename):
    with open(filename, 'r') as file:
        return json.load(file)

input_file = "/content/drive/MyDrive/Final year Pro/video-transcript.json"

data = read_json_file(input_file)

for item in data:
    item["start"] = format_seconds(item["start"])
    item["end"] = format_seconds(item["end"])

output_file = "converted_data.json"
with open(output_file, "w") as outfile:
    json.dump(data, outfile, indent=4)

print("Updated data with new formatted timestamps has been saved to", output_file)


Updated data with new formatted timestamps has been saved to converted_data.json


In [2]:
import json

# Function to read JSON data from file
def read_json_file(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Read captions JSON data from file
captions_file = "/content/drive/MyDrive/Final year Pro/video-captions.json"
captions_data = read_json_file(captions_file)

# Read transcript JSON data from file
transcript_file = "/content/converted_data.json"
transcript_data = read_json_file(transcript_file)

# Iterate over each transcript entry
for item in transcript_data:
    start_time = item["start"]
    end_time = item["end"]

    # Find captions within the duration
    captions_within_duration = {}
    for frame_timestamp, caption_text in captions_data.items():
        frame_number = int(frame_timestamp.split("_")[1].split(".")[0])
        if int(start_time) <= frame_number <= int(end_time):
            captions_within_duration.setdefault(frame_timestamp,caption_text)


    # Merge captions and append to transcript entry
    # merged_caption = " ".join(captions_within_duration)
    # item["merged_caption"] = merged_caption
    item["image_captions"]= captions_within_duration

# Write the updated transcript data to a new JSON file
output_file = "updated_transcript.json"
with open(output_file, "w") as outfile:
    json.dump(transcript_data, outfile, indent=4)

print("Updated transcript data with merged captions has been saved to", output_file)


Updated transcript data with merged captions has been saved to updated_transcript.json


In [11]:
import json

# Function to read JSON data from file
def read_json_file(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Read captions JSON data from file
captions_file = "/content/drive/MyDrive/Final year Pro/video-captions.json"
captions_data = read_json_file(captions_file)

# Read transcript JSON data from file
transcript_file = "/content/converted_data.json"
transcript_data = read_json_file(transcript_file)

# Iterate over each transcript entry
for item in transcript_data:
    start_time = item["start"]
    end_time = item["end"]

    # Find captions within the duration
    merged_caption = ""
    for frame_timestamp, caption_text in captions_data.items():
        frame_number = int(frame_timestamp.split("_")[1].split(".")[0])
        if int(start_time) <= frame_number <= int(end_time):
            merged_caption += caption_text + " "

    # Update transcript entry with merged caption
    item["merged_caption"] = merged_caption.strip()

# Write the updated transcript data to a new JSON file
output_file = "transcripts.json"
with open(output_file, "w") as outfile:
    json.dump(transcript_data, outfile, indent=4)

print("Updated transcript data with merged captions has been saved to", output_file)


Updated transcript data with merged captions has been saved to transcripts.json
